# ВВЕДЕНИЕ

Язык SQL. Диалекты в DBMS (database management system):
    
    -Oracle (высоконагруженные приложения, проприетарная)
    -MySQL (веб-сайты, свободная)
    -PostgreSQL (пост-реляционная, например, может хранить в одной ячейке несколько значений)
    -MS SQL (сервера на базе винды)
    -SQLite (простая, свободная, для локальных приложений)
    -Access (альтернатива SQLite от микрософт)

Выборка данных SELECT WHERE

    -Варианты условий
    -Выборка конкретных полей
    -Условия с подзапросом

Добавление записей INSERT INTO

    -Вставка одной записи
    -Вставка нескольких
    -Заполнение только указанных полей
    -Выборка вставленных записей (SELECT LAST_INSERT_ID в MySQL), не сработает при одновременной вставке нескольких записей

Обновление данных UPDATE WHERE

    -Условия аналогично SELECT
    -Нельзя делать подзапросы в условиях, если обновляется эта же таблица

Удаление данных DELETE
    
    -Синтаксис аналогичен SELECT

Агрегация данных

Примеры информационных потребностей:

    -Суммировать значения по указанному атрибуту для записей с определенными условиями
    -Статистика в разрезе значений указанного атрибута


# Реляционная модель данных

Сложно и просто

    Математический аппарат
    Простые формализмы

Структурная часть

    Набор понятий, родовая структура

Манипуляционная часть
    
    Операции над данными

Ограниченияцелостности
    
    Средства поддержания структуры и содержимого БД в соответствии модели данных

DML = data modification language (SELECT, INSERT, UPDATE, DELETE, etc)

# Архитектура ANSI SPARC

Определяет принцип организации СУБД (необязательно РМД)  

Цель: введение уровней абстракции для пользователей и архитекторов БД
    
    Разные группы пользователей ИC должны иметь разные представления данных
    Пользователи должны быть абстрагированы от физической и логической организации данных
    Администраторы БД должны иметь возможность вносить изменения в схему данных без учета особенностей физического хранения данных
    Внутренняя структура БД не должна зависеть от физических аспектов хранения информации

# SQL/DDL

    ● Создание схемы
    ● Создание доменов
    ● Создание таблиц
    ● Изменение таблиц

SQL = DDL + DML + DCL (операторы для администрирования БД) + TCL (операторы для работы с транзакциями)  
  
DDL (data definition language) CREATE, ALTER, DROP, RENAME и др.   
DML (data manipulation language) SELECT, INSERT, UPDATE, DETELE и др.




## SQL/DDL. Создание схемы. Синтаксис, пример

    ● CREATE SCHEMA 
    <schema definition> ::= CREATE SCHEMA <schema name clause> 
            [ <schema character set specification> ]     
            [ <schema element>... ]
            
    ● Пример
    CREATE SCHEMA IF NOT EXISTS `store` DEFAULT CHARACTER SET utf8;



## SQL/DDL. Создание домена. Синтаксис, пример

    ●CREATE DOMAIN
    domain_definition ::= CREATE DOMAIN domain_name [AS] data_type    
            [ default_definition ]    
            [ domain_constraint_definition_list ]
    default_definition ::= DEFAULT { literal | niladic_function | NULL }
    domain_constraint_definition_list ::= [CONSTRAINT constraint_name]
        CHECK (conditional_expression)
    
    ●Пример (PostgreSQL) в mysql нет понятия домена (исторические причины, начиналось как простая СУБД, вместо домена надо вводить таблицу-словарь)
    CREATE DOMAIN SALE_STATUS AS VARCHAR(45)    
        DEFAULT 'new'    
        CHECK (VALUE IN ('new', 'process', 'assembly', 'ready', 'delivering','issued', 'rejected'))    
    CONSTRAINT SALE_STATUS_NOT_NULL CHECK (VALUE IS NOT NULL);      -- например так можно добавить именованное ограничение неравности NULL 

## SQL/DDL. Создание таблицы. Варианты синтаксиса

    ●CREATE TABLE
    base_table_definition ::= CREATE TABLE base_table_name (base_table_element_commalist)

    base_table_element ::= column_definition | base_table_constraint_definition         -- описание элементов таблицы

    column_definition ::= column_name                                                   -- описание столбцов
            { data_type | domain_name }                                                 -- тип данны/домен данных
            [ default_definition ]                                                      -- занчения по умолчанию
            [ column_constraint_definition_list ]                                       -- ограничения на занчения в столбцах

    ●CREATE TABLE

    column_constraint_definition ::=                                                    -- ограничения могут включать
            [ CONSTRAINT constraint_name ]                                              -- могут быть именованными (для использования в других таблицах)
            NOT NULL                                                                    -- не пусто
            | { PRIMARY KEY | UNIQUE }                                                  -- является ключом/уникальным
            | references_definition                                                     -- сслыки на другие таблицы (или этой же таблицы)
            | CHECK ( conditional_expression )                                          -- проверки условий

    references_definition ::=    
        REFERENCES base_table_name [ (column_commalist) ]        
                [ MATCH { SIMPLE | FULL | PARTIAL } ]                                   -- тип ссылки/совпадения значения (SIMPLE по умолчанию)
                [ ON DELETE referential_action ]                                        -- что должно происходить при удалении/изменении (для поддержки ссылочной целостности)
                [ ON UPDATE referential_action ]

    ●CREATE TABLE
    base_table_constraint_definition ::=    
        [ CONSTRAINT constraint_name ]    
        { PRIMARY KEY | UNIQUE } ( column_commalist )                                   -- можем указать поля, которые составляют ключ
        | FOREIGN KEY ( column_commalist )       
        
    references_definition    
        | CHECK ( conditional_expression )
        
    referential_action ::=   
        { NO ACTION | RESTRICT | CASCADE | SET DEFAULT | SET NULL }


Создание таблицы. Примеры

    CREATE TABLE IF NOT EXISTS `store`.`category` (
    `id` INT NOT NULL,
    `name` VARCHAR(255) NULL,       -- до 255 символов или NULL
    PRIMARY KEY (`id`))             -- ключ
    ENGINE = InnoDB                 -- распространыенный движ.
    DEFAULT CHARACTER SET = utf8    
    COLLATE = utf8_general_ci;      -- набор правил для сравнения срок ><= (нужен не для всех СУБД)


    CREATE TABLE IF NOT EXISTS `store`.`category_has_good` (
        `category_id` INT NOT NULL, `good_id` INT NOT NULL,
        PRIMARY KEY (`category_id`, `good_id`),                             -- составной ключ
        INDEX `fk_category_has_good_good1_idx` (`good_id` ASC),             -- индекс 
        INDEX `fk_category_has_good_category_idx` (`category_id` ASC),
        CONSTRAINT `fk_category_has_good_category`
            FOREIGN KEY (`category_id`) REFERENCES `store`.`category` (`id`)    -- на что ссылается
            ON DELETE NO ACTION ON UPDATE NO ACTION,                            -- что делать при удалении/изменении кортежа таблицы
        CONSTRAINT `fk_category_has_good_good1`
            FOREIGN KEY (`good_id`) REFERENCES `store`.`good` (`id`)
            ON DELETE NO ACTION ON UPDATE NO ACTION)
    ENGINE = InnoDB
    DEFAULT CHARACTER SET = utf8
    COLLATE = utf8_general_ci;

Индекс (англ. index) — объект базы данных, создаваемый с целью повышения производительности поиска данных.  
По умолчанию многие СУБД создают индекс по полю, обозначенному как ключ (первичный или внешний). Ускорение работы с использованием индексов достигается в первую очередь за счёт того, что индекс имеет структуру, оптимизированную под поиск — например, сбалансированного дерева.  


## SQL/DDL. Изменение таблицы

    ● ALTER TABLE                                               -- позволяет
    base_table_alteration ::= ALTER TABLE base_table_name
        column_alteration_action                                -- изменить/удалить столбец
        | base_table_constraint_alternation_action              -- изменить/удалить ограничение

    column_alteration_action ::=                                -- изменение столбца
            ADD [ COLUMN ] column_definition
        | ALTER [ COLUMN ] column_name
            { SET default_definition | DROP DEFAULT }
        | DROP [ COLUMN ] column_name
            { RESTRICT | CASCADE }

Примеры
    
    ALTER TABLE `store`.`sale`
        ADD COLUMN
            `is_exclusive_case` BOOLEAN NOT NULL DEFAULT 0;     -- добавить колонку

    ALTER TABLE `store`.`sale`
        DROP COLUMN dt_created,                                 -- удалить поля
        DROP COLUMN dt_modified,
        DROP FOREIGN KEY fk_order_status1,                      -- удалить ключ
        DROP COLUMN status_id,
        ADD COLUMN ts_modified TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,   -- добавить поле типа таймстемп
        ADD COLUMN sale_status VARCHAR(45) NOT NULL DEFAULT 'new'                                       -- добавить поле и к нему    
        CHECK (VALUE IN ('new', 'process', 'assembly', 'ready', 'delivering', 'issued', 'rejected'));   -- аналог домена в майскуле, проверка органичений

Изменение органичений столбца

    ALTER TABLE
    base_table_constraint_alternation_action ::=
        ADD [ CONSTRAINT ] base_table_constraint_definition
        | DROP CONSTRAINT constraint_name
        { RESTRICT | CASCADE }

● Пример

    ALTER TABLE `store`.`sale`
        DROP FOREIGN KEY fk_order_client1;
    
    ALTER TABLE `store`.`sale`
        DROP INDEX fk_order_client1_idx;
    
    ALTER TABLE `store`.`sale`
        DROP COLUMN client_id;

# Нормализация баз данных

    ННФ - ненормальная форма - без требований
    1НФ - первая нормальная форма - обязательное требование
    реляционных БД
    2НФ, 3НФ, НФБК - устранение часто возникающих аномалий
    4НФ - решение проблем с многозначной зависимостью
    5НФ - решение проблем с зависимостями соединения

## Первая нормальная форма  (убираем в ННФ многозначность)

    Ненормализованная форма - отношение содержит повторяющиеся группы
    данных в одной ячейке.
    Первая нормальная форма (1НФ) - отношение, в котором на пересечении
    строки и столбца содержится только одно значение

    Задача              Исполнитель
    Нормализовать БД    Иванов
    Нормализовать БД    Петров
    Составить ER-модель Иванов

## Вторая нормальная форма (убираем в 1НФ составные ключи (наборы полей, функционально определющие остальные поля))

    Отношение находится в 1НФ.
    Каждый атрибут, не входящий в первичный ключ,полностью
    функционально зависит от первичного ключа.

Если у вас первичный ключ только один - id, то отношение уже в 2НФ.  
Составной первичный ключ возникает при декомпозиции в 1НФ связей многие-ко-многим.  
NB первичный ключ, этот тот потенциальный ключ (набор атрибутов, определяющий остальные атрибуты), который мы выбрали основным.

1НФ:

    Модель 	    Фирма 	Цена 	    Скидка
    M5          BMW 	5500000 	5%
    X5M 	    BMW 	6000000 	5%
    M1          BMW 	2500000 	5%
    GT-R 	    Nissan 	5000000 	10%

2НФ:

    Модель 	    Фирма 	Цена 	    
    M5          BMW 	5500000 	
    X5M 	    BMW 	6000000 	
    M1          BMW 	2500000 	
    GT-R 	    Nissan 	5000000 	

    Фирма 	Скидка
    BMW 	5%
    Nissan 	10%



## Третья нормальная форма (убираем транзитивные зависимости)

    Находится во 2НФ.
    Нет атрибутов, не входящих в первичный ключ, которые находятся в транзитивной зависимости от первичного ключа.

Все атрибуты зависят только от первичного ключа напрямую.   
3НФ: Любой неключевой атрибут зависит непосредственно от ключа. + 2НФ (то есть зависит полностью).

2НФ:

    Модель 	Магазин 	Телефон
    BMW 	Риал-авто 	87-33-98        // ключ таблицы - поле Модель, а телефон определяется полем Магазин
    Audi 	Риал-авто 	87-33-98        // здесь зависимость Модель -> Телефон транзитивная через поле Магазин
    Nissan 	Некст-Авто 	94-54-12

3НФ:

    Магазин 	Телефон
    Риал-авто 	87-33-98
    Некст-Авто 	94-54-12
    
    Модель 	Магазин
    BMW 	Риал-авто
    Audi 	Риал-авто
    Nissan 	Некст-Авто

## Нормальная форма Бойса-Кодда

    Находится в 3НФ
    Каждый детерминант является потенциальным ключом.

Найти все детерминанты и убедиться, что они являются потенциальными ключами, то есть однозначно определяют запись в таблице.  
Если есть непотенциальные ключи, то вынести в отдельную сущность.Нормальная форма Бойса-Кодда  
НФБК: Убирать лишние ключи: в таблице должен остаться только один первичный ключ.  

3НФ: 

    Номер стоянки 	Время начала 	Время окончания 	Тариф
    1 	09:30 	10:30 	Бережливый      // здесь тарифы привязаны к номеру стоянки, но неявно, поэтому возможны ошибки (приписать не тот тариф к не той стоянке)
    1 	11:00 	12:00 	Бережливый      // поэтому тариф также является ключом (детерминант, т.е. определяет стоянку), но данная таблица этого не отражает, хотя и является 3НФ
    1 	14:00 	15:30 	Стандарт
    2 	10:00 	12:00 	Премиум-В
    2 	12:00 	14:00 	Премиум-В
    2 	15:00 	18:00 	Премиум-А

БКНФ:

    Тариф 	Номер стоянки 	Имеет льготы
    Бережливый 	1 	Да
    Стандарт 	1 	Нет
    Премиум-А 	2 	Да
    Премиум-В 	2 	Нет

    Тариф 	Время начала 	Время окончания
    Бережливый 	09:30 	10:30
    Бережливый 	11:00 	12:00
    Стандарт 	14:00 	15:30
    Премиум-В 	10:00 	12:00
    Премиум-В 	12:00 	14:00
    Премиум-А 	15:00 	18:00


## Многозначная зависимость

Для каждого значения атрибута А есть наборы значений атрибутов В и С, но входящие в эти наборы значения не зависят от В и С.  
Тривиальная многозначная зависимость - атрибут В является подмножеством А или A⋃B - все множество отношения.  
Нетривиальная - не выполняются условия.  

Сложность именно с нетривиальными зависимостями.

Предположим, что рестораны производят разные виды пиццы, а службы доставки ресторанов работают только в определенных районах города.   
Составной первичный ключ соответствующей переменной отношения включает три атрибута: {Ресторан, Вид пиццы, Район доставки}.  
Тут многозначная зависимость:  
    
    {Ресторан} → {Вид пиццы}
    {Ресторан} → {Район доставки}

То есть, например, при добавлении нового вида пиццы придется внести по одному новому кортежу для каждого района доставки.   
Возможна логическая аномалия, при которой определенному виду пиццы будут соответствовать лишь некоторые районы доставки из обслуживаемых рестораном районов.  

## Четвертая нормальная форма (убираем многозначные зависимости)

Для предотвращения аномалии нужно декомпозировать отношение, разместив независимые факты в разных отношениях.   
В данном примере следует выполнить декомпозицию на {Ресторан, Вид пиццы} и {Ресторан, Район доставки}.




# Прочие нормальные формы 5НФ, ДКНФ, 6НФ

## 5НФ

Например есть таблица, кот. содержит три атрибута «Поставщик», «Товар» и «Покупатель»  
Не должны налагаться органичения в такого рода примерах на то, какой покупатель какой товар сможет приобрести у какого поставщика.  
Такие таблицы надо разносить в разные отношения. А значит у нас будет 3 отношения, которые будут связаны. Извлечение из них информации зависит от соединения этих отношений.  
И если попробовать извлечь информацию из каких-то двух из трех таблиц, возможно извлечение некорректной информации.

Пятая нормальная форма ориентирована на работу с зависимыми соединениями. Указанные зависимые соединения между тремя атрибутами встречаются очень редко. Зависимые соединения между четырьмя, пятью и более атрибутами указать практически невозможно.  

## ДКНФ Доменно-ключевая нормальная форма

Переменная отношения находится в ДКНФ тогда и только тогда, когда каждое наложенное на неё ограничение является логическим следствием ограничений доменов и ограничений ключей, наложенных на данную переменную отношения.
Ограничение домена – ограничение, предписывающее использовать для определённого атрибута значения только из некоторого заданного домена. Ограничение по своей сути является заданием перечня (или логического эквивалента перечня) допустимых значений типа и объявлением о том, что указанный атрибут имеет данный тип.  

## 6НФ

Самая строгая форма. Означает, что никакое отношение в БД не может быть декомпозировано без потери информации.  
Деконпозировано все что можно.  Полезная для хронологических баз данных, где максимально возможная декомпозиция позволяет бороться с избыточностью и упрощает поддержание целостности базы данных.




# Использование ORM

Объектно-Реляционное Отображение (Object-Relational Mapping) - технология, связывающая модель базы данных и концепции ООП.   
Позволяет работать с данными как с объектами.

    from django.db import models

    class Category(models.Model):
        name = models.CharField(max_length=100)                         # атрибуты отношений (таблиц), тип данных, органичение на длину

        def get_aviable_products(self):                                 # метод класса, позволяет получить продукты в БД с количеством >0
            return Product.objects.filter(category=self, quantity__gte=0)

    class Product(models.Model):
        name = models.CharField(max_length=100)
        price = models.DecimalField(max_digits=10, decimal_places=2)    # тип данных лучше подходит чем с плавающей точкой
        quantity = models.IntegerField(default=0)                       # можно задать значение по умолчанию
        category = models.ForeignKey(Category, null=True)               # внешний ключ в отношении 


## Миграции

●Упрощают работу со схемой базы  
●Позволяют быстро переключаться между разными вариантами схемы (в том числе откатывать изменения)

Миграция - механизм, автоматизирующий перенос БД

### Django

manage.py makemigrations - создает непривязанные к СУБД инструкции для отображения текущего состояния моделей   
manage.py migrate - используя миграции приводит базу данных, указанную в settings.py к виду актуальному models.py

## Онлайн-магазин

    Товар: название, цена, количество, категории
    Категория: название
    Пользователь: почта, пароль
    Корзина: пользователь, товары, итоговая сумма

Нормализация - добавляются промежуточные таблицы многие-ко-многим ПродуктКатегория ПродуктКорзина из id продуктов/категорий/корзин, итого 6 таблиц


    from django.db import models
    from django.contrib.auth.models import User                             # отношение Пользователь (встроена в ОРМ)

    class Category(models.Model):                                           # отношение Категория
        name = models.CharField(max_length=100)

    class Product(models.Model):                                            # отношение Товар
        name = models.CharField(max_length=100)
        price = models.DecimalField(max_digits=10, decimal_places=2)
        quantity = models.IntegerField(default=0)            
        category = models.ManyToManyField(Category)                         # самостоятельно создает промежуточную таблицу

    class Basket(models.Model):                                             # отношение Корзина
        user = models.OneToOneField(User)                                   # создает внешний ключ в флагом Уникальный
        products = models.ManyToManyField(Category, null=True)              # еще одна промежуточная таблица неявно заданы тут 

        total_sum = models.DecimalField(max_digits=10, decimal_places=2)

    # $ django-admin startproject online_store_example

    # settings.py -> INSTALLED_APPS = [...    'online_store_example',]      указание, задействовать данный файл models.py
    # $ ./manage.py makemigrations online_store_example                     создает непривязанные к СУБД инструкции для отображения текущего состояния моделей
    # $ ./manage.py migrate                                                 используя миграции приводит базу данных, указанную в settings.py к виду актуальному models.py

    # $ sqlite
    # sqlite> чето типа .schema online_store_example_product_category       но у меня не сработало    
    # вместо этого можно посмотреть в DB viewer:
    # CREATE TABLE "online_store_example_product_category" (
    #       "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, 
    #       "product_id" bigint NOT NULL REFERENCES "online_store_example_product" ("id") DEFERRABLE INITIALLY DEFERRED, 
    #       "category_id" bigint NOT NULL REFERENCES "online_store_example_category" ("id") DEFERRABLE INITIALLY DEFERRED)

    # короче вообще не так, как в уроке -> все хуйня, всех расстрелять


# Сложность выполнения запросов. EXPLAIN

## Стратегия составления плана

●С какой таблицы начать: выбирает ту, где меньше кортежей.  
●Какие условия лучше обработать первыми, чтобы отфильтровать больше.  
●Какие ключи использовать, чтобы как можно раньше как можно больше отфильтровать.

# Индексы

Быстрее всего искать данные в упорядоченном списке  
Чтобы изменять данные в упорядоченном списке можно использовать дерево

    Кластерный индекс:
    ● Упорядоченное дерево, на листьях которого находятся сами данные (м.б. 1 на таблицу)
    Некластерные индексы:
    ● Структуры, выстраиваемые вокруг данных со ссылками на кортежи


Основные виды  

    В-дерево
        Подходит для:
        Операций сравнения: <, <=, =, >=, >
        Проверки is (not) null
        Сортировки
        Группировки
        
    Хэш-индекс
        Подходит для:
        Сравнений = и !=
        WHERE field_name IN (...)
        Используется во внешних ключах.
    
    Другие типы индексов
        Геолокация
        Перекрывающие индексы
        Многомерные структуры
        Битовые индексы

## Индексы и план запроса
    
    Индекс создает ключ, который СУБД может использовать в плане запроса.
    СУБД выбирает наиболее подходящий ключ среди доступных, но пользователь может указать, какой ключ использовать лучше.

# Нереляционные СУБД

## Причины возникновения NoSQL

    Потребность в распределенных СУБД
    Потребность в быстрой работе с данными
    Некоторые часто встречаемые задачи можно моделировать проще
    
## Особенности NoSQL

    Объект данных - более сложная структура, чем просто у строки в
    таблице
    Без строго определенной схемы
    Без операций соединения JOIN
    Масштабируется
    Нет SQL, но бывает что-то похожее

## CAP-теорема

    ● Consistency - информация на разных узлах согласована
    ● Availability - система отвечает на запросы
    ● Partition tolerance - связи между узлами могут обрываться

    Теорема: для распределенных систем можно выбрать только 2 свойства

## BASE-архитектура

    Basically Available - сбой узла приводит к отказу только для части
    пользователей
    Soft-state - система может находиться в неусточивом состоянии
    Eventual Consistency - когда-нибудь согласуется
    
## Классы NoSQL

    Ключ-значение - быстрый доступ к данным по ключу
    Документоориентированные - удобство моделирования вложенных
    структур без жесткой схемы
    Семейство столбцов - таблицы с множеством незаполненных
    атрибутов (разреженная матрица)
    Классы могут пересекаться.
    
## Ключ-значение
    
    Используется для:
        Работы данными в реальном времени
        Кэширования результатов долгих операций
        Файловые системы на основе “ключ-значение”

Примеры СУБД: Berkeley DB, Redis, Memcache

## Документоориентированные

    Используются для:
        Хранения документов и поиска по ним
        Информационные сервисы
        Данные с нечеткой схемой
        Примеры: MongoDB, CouchDB

## Семейство столбцов
    
    Используется для:
        Хранение данных, собранных с веб-страниц
        Хранение и поиск по большому объему данных
        Примеры: Cassandra, HBase


# Redis

    ● 100k операций в секунду для одного узла
    ● Всего больше сотни команд
    ● Все команды работы с данными по крайне мере содержат ключ
    ● Для каждого типа данных свой набор CRUD-команд, различаются префиксами, например: LPOP (list), SPOP (set)
    

## Типы данных Redis



Ключи: уникальная строка для одного набора данных  
Значения:

    ● Строки - скалярные значения, для разных подтипов есть разные операции
    ● Хэши - для хэш-таблиц внутри значений
    ● Списки - как list в Python с теми же операциями
    ● Множества - как set в Python с теми же операциями
    ● Упорядоченные множества

## Репликация в Redis

● Обновления поступают не только от мастера, но и от уже обновленных слейвов, децентрализуя операцию  
● Изменения всегда просты (с одним ключом), поэтому передаются значения, а не операторы

## Подписки (PubSub)

Не нужно каждый раз спрашивать, есть ли новые данные, сервер сам пройдется по активным подписчикам и отправит им изменения, когда они появятся.

# Особенности MongoDB

    ● Самая популярная ⇒ самая стабильная
    ● Простота хранения данных ⇒ нет проблем с миграциями
    ● Сложность восприятия запросов, но легко писать интерпретаторы и генераторы ⇒ разнообразие ORM


## Пользовательские команды  

Все параметры операций - валидный JSON:

    ● Сложность восприятия: скобки, нет многих символов
    ● Легко найти драйвер с подходящим синтаксисом в любом языке

    db.unicorns.find({gender: 'f', $or: [{loves: 'apple'}, {loves: 'orange'}, {weight: {$lt: 500}}]})

## Что есть вместо JOIN

    ● Ссылки и списки ссылок на объекты
    ● Встроенные документы, денормализация
    ● Тип DBRef для драйверов

## Ссылки на объекты

Но нужно делать несколько запросов

## Встроенные документы

Аномалии обновления, если данные дублируются

## DBRef

Только для использования в драйверах и ORM

db.post.find({'author': DBRef("user", ObjectId('...'))})

Репликация похожа на Redis - от мастер-узла обновления распространяются слейвам, от них их слейвам и т.д.  
В отличие от Redis при падении мастер-узла MongoDB выбирает новый мастер-узел, который будет принимать запросы на обновление и рассылать слейвам.  
    
    * если мастер-узел оживет, то станет слейвом, с возможностью стать мастером если вдруг упадет новый
    ** Redis в этом случае потеряет возможность обработки обновлений данных.